# Preparação

In [ ]:
# Bibliotecas
import pandas as pd
import numpy as np
import sys
import chardet
import chardet.universaldetector
!pip install unidecode
import unidecode
from functools import lru_cache
import os
from os import remove
import gc
!pip install wget
import wget
!pip install py7zr
from py7zr import SevenZipFile
import glob
from io import StringIO

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Ajustando diretórios de trabalho
diretorio_brutos = "..."

## Funções

In [ ]:
# Função para baixar e extrair os arquivos da CAGED
def extract_caged (ano, mes):
  """ Faz o download dos arquivos do NOVO CAGED para o ano e o mês especificados.
  O período do NOVO CAGED tem início em Janeiro de 2020.
  Nesse script não extraímos os microdados antriores a esse período devido a divergências metodológicas estabelecidas pelo MTE.

  São 3 tipos de arquivo: CAGEDMOV, CAGEDFOR e CAGEDEXC.
    1. CAGEDMOV: movimentações declaradas dentro do prazo com competência de declaração igual a AAAAMM.
    2. CAGEDFOR: movimentações declaradas fora do prazo com competência de declaração igual a AAAAMM.
    3. CAGEDEXC: movimentações excluídas com competência de declaração da exclusão igual a AAAAMM.

  Argumentos:
      ano: Ano do arquivo.
      mes: Mês do arquivo.
  """
  # Download
  if ano < 2020:
        print(f"Os microdados de {mes}/{ano} não serão baixados.")
        return

  tipos_arquivos = ['CAGEDEXC', 'CAGEDFOR', 'CAGEDMOV']

  for tipo in tipos_arquivos:
    url = f'ftp://ftp.mtps.gov.br/pdet/microdados/NOVO CAGED/{ano}/{ano}{mes}/{tipo}{ano}{mes}.7z'
    nome_arquivo = f'{tipo}{ano}{mes}.7z'

    try:
      print(f'\nBaixando {tipo} de {mes}/{ano}...')
      wget.download(url, nome_arquivo)
      print(f'\n{tipo}{ano}{mes}.7z baixado com sucesso.')
    except:
      print(f'\nOs microdados {tipo} de {mes}/{ano} não estão disponíveis.')
      continue

    # Extração do arquivo 7z
    try:
        with SevenZipFile(nome_arquivo, mode = 'r') as archive:
             archive.extractall(diretorio_brutos)
             print(f'{tipo}{ano}{mes}.txt extraído com sucesso.')
    except:
        print(f'Erro ao extrair {tipo}{ano}{mes}.7z.')
        continue

    # Remoção do arquivo 7z após a extração
    try:
        remove(nome_arquivo)
        print(f'{nome_arquivo} removido.\n')
    except:
        print(f'Erro ao remover {nome_arquivo}.')

# Download e Extração dos arquivos

## Extração (2020 até mais recente)

In [ ]:
# Extraindo arquivos para todo o período

# Definir meses de janeiro a dezembro
meses = [f'{i:02}' for i in range(1, 13)]

# Loop
for ano in range(2020, 2026):
    for mes in meses:
        extract_caged(ano, mes)

## Extração (personalizada)

In [ ]:
# Extraindo arquivos de um mês específico
extract_caged(2023, 11)
extract_caged(2025, '01')